In [2]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Border, Side, PatternFill, Alignment
from openpyxl.worksheet.page import PageMargins

# Load and split data
df = pd.read_csv('picking.csv')
split_cols = df['LW SKU'].str.split('-', n=2, expand=True)
split_cols.columns = ['SKU Part 1', 'SKU Part 2', 'SKU Part 3']
df_formatted = pd.concat([split_cols, df['Qty ordered']], axis=1)

# Create workbook
wb = Workbook()
ws = wb.active
ws.title = "Formatted"

# Write data
for r in dataframe_to_rows(df_formatted, index=False, header=True):
    ws.append(r)

# Styling
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

dark_grey_fill = PatternFill(start_color="A9A9A9", end_color="A9A9A9", fill_type="solid")
# light_grey_fill = PatternFill(start_color="D3D3D3", end_color="D3D3D3", fill_type="solid")
# dark_grey_fill = PatternFill(start_color="808080", end_color="808080", fill_type="solid")
center_alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

# Apply borders, alignment, conditional fill to all rows including header
for row in ws.iter_rows(min_row=1, max_row=ws.max_row):
    for cell in row:
        cell.border = thin_border
        cell.alignment = center_alignment
    if row[0].row > 1:
        qty_cell = row[3]
        if qty_cell.value != 1:
            qty_cell.fill = dark_grey_fill

# Merge function
def merge_column(ws, col_idx, start_row, end_row):
    merge_start = start_row
    while merge_start <= end_row:
        current_value = ws.cell(row=merge_start, column=col_idx).value
        merge_end = merge_start
        while (merge_end + 1 <= end_row and
               ws.cell(row=merge_end + 1, column=col_idx).value == current_value):
            merge_end += 1
        if merge_end > merge_start:
            ws.merge_cells(start_row=merge_start, start_column=col_idx,
                           end_row=merge_end, end_column=col_idx)
            ws.cell(row=merge_start, column=col_idx).alignment = center_alignment
        merge_start = merge_end + 1

# Merge SKU Part 1 and Part 2
merge_column(ws, 1, 2, ws.max_row)
row = 2
while row <= ws.max_row:
    val = ws.cell(row=row, column=1).value
    group_start = row
    while row + 1 <= ws.max_row and ws.cell(row + 1, column=1).value == val:
        row += 1
    group_end = row
    merge_column(ws, 2, group_start, group_end)
    row += 1

# Auto-fit columns
for col in ws.columns:
    max_length = 0
    col_letter = col[0].column_letter
    for cell in col:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))
    ws.column_dimensions[col_letter].width = max_length + 2

# Set print margins
ws.page_margins = PageMargins(
    left=0.5, right=0.5, top=0.5, bottom=0.5,
    header=0.5, footer=0.5
)

# Save
wb.save("formatted_test.xlsx")
